In [1]:
import os
import fiona
import hashlib
import boto3
import json
import datacube
from datacube.api import GridWorkflow
from madmex.wrappers import gwf_query
from madmex.models import SegmentationInformation
from madmex.util import parser_extra_args
from madmex.settings import TEMP_DIR
from importlib import import_module
import gc

In [51]:
name_of_product = 's2_10m_fiona_test_2_2018'
gwf_kwargs = {'region': 'Jalisco', 'begin': '2018-01-01', 'end':'2018-12-31'}
gwf_kwargs.update(product=name_of_product)

In [52]:
iterable = gwf_query(**gwf_kwargs)

In [53]:
lista_iter = list(iterable)

In [54]:
[lista_iter[x][0] for x in range(0,len(lista_iter))]

[(28, -27),
 (28, -28),
 (30, -29),
 (30, -30),
 (28, -29),
 (28, -30),
 (29, -30),
 (30, -27),
 (29, -29),
 (30, -28),
 (29, -28),
 (29, -27)]

In [55]:
var = lista_iter[8]

In [56]:
algorithm = 'bis' #'slic'
datasource = 'sentinel2'
name = 's2_10m_some_scenes_fiona_test'
year = '2018'
extra_args = parser_extra_args(['t=30','s=0.5','c=0.7']) #for slic: parser_extra_args(['n_segments=10000', 'compactness=0.5'])

In [57]:
band_list = ['ndvi_mean']

In [58]:
try:
    module = import_module('madmex.segmentation.%s' % algorithm)
    Segmentation = module.Segmentation
except ImportError as e:
    raise ValueError('Invalid model argument')

In [59]:
meta, _ = SegmentationInformation.objects.get_or_create(
    algorithm=algorithm, datasource=datasource,
    parameters=json.dumps(extra_args),
    datasource_year=year,
    name=name)

In [60]:
meta

<SegmentationInformation: SegmentationInformation object (1)>

In [61]:
geoarray = GridWorkflow.load(var[1], measurements=band_list)

In [62]:
dataset_name = meta.datasource + '_%d_%d_' % (var[0][0], var[0][1]) + meta.datasource_year

In [63]:
dataset_name

'sentinel2_29_-29_2018'

In [64]:
hash = hashlib.md5(dataset_name.encode('utf-8')).hexdigest()[0:6]

In [65]:
name_file = hash + '_' + dataset_name

In [66]:
name_file

'dc2f9b_sentinel2_29_-29_2018'

In [67]:
path = os.path.join(TEMP_DIR, 'segmentation_results')

In [68]:
path

'/shared_volume/temp/segmentation_results'

In [69]:
if not os.path.exists(path):
    os.makedirs(path)

In [70]:
seg = Segmentation.from_geoarray(geoarray, **extra_args)

In [71]:
seg.geobox.extent.wkt

'POLYGON ((2427740 1026160,2427740 976140,2477760 976140,2477760 1026160,2427740 1026160))'

In [72]:
seg.geobox.extent.json

{'type': 'Polygon',
 'coordinates': [[(2427740.0, 1026160.0),
   (2427740.0, 976140.0),
   (2477760.0, 976140.0),
   (2477760.0, 1026160.0),
   (2427740.0, 1026160.0)]]}

In [73]:
try:
    seg.segment()
    fc = seg.polygonize()
    shp_path = seg.to_shapefile(filename=name_file, fc=fc)
    gc.collect()
except Exception as e:
        print(e)